--- Day 19: Medicine for Rudolph ---

Rudolph the Red-Nosed Reindeer is sick! His nose isn't shining very brightly, and he needs medicine.

Red-Nosed Reindeer biology isn't similar to regular reindeer biology; Rudolph is going to need custom-made medicine. Unfortunately, Red-Nosed Reindeer chemistry isn't similar to regular reindeer chemistry, either.

The North Pole is equipped with a Red-Nosed Reindeer nuclear fusion/fission plant, capable of constructing any Red-Nosed Reindeer molecule you need. It works by starting with some input molecule and then doing a series of replacements, one per step, until it has the right molecule.

However, the machine has to be calibrated before it can be used. Calibration involves determining the number of molecules that can be generated in one step from a given starting point.

For example, imagine a simpler machine that supports only the following replacements:

H => HO  
H => OH  
O => HH  

Given the replacements above and starting with HOH, the following molecules could be generated:

    HOOH (via H => HO on the first H).
    HOHO (via H => HO on the second H).
    OHOH (via H => OH on the first H).
    HOOH (via H => OH on the second H).
    HHHH (via O => HH).

So, in the example above, there are 4 distinct molecules (not five, because HOOH appears twice) after one replacement from HOH. Santa's favorite molecule, HOHOHO, can become 7 distinct molecules (over nine replacements: six from H, and three from O).

The machine replaces without regard for the surrounding characters. For example, given the string H2O, the transition H => OO would result in OO2O.

Your puzzle input describes all of the possible replacements and, at the bottom, the medicine molecule for which you need to calibrate the machine. How many distinct molecules can be created after all the different ways you can do one replacement on the medicine molecule?


--- Part Two ---

Now that the machine is calibrated, you're ready to begin molecule fabrication.

Molecule fabrication always begins with just a single electron, e, and applying replacements one at a time, just like the ones during calibration.

For example, suppose you have the following replacements:

e => H
e => O
H => HO
H => OH
O => HH

If you'd like to make HOH, you start with e, and then make the following replacements:

    e => O to get O
    O => HH to get HH
    H => OH (on the second H) to get HOH

So, you could make HOH after 3 steps. Santa's favorite molecule, HOHOHO, can be made in 6 steps.

How long will it take to make the medicine? Given the available replacements and the medicine molecule in your puzzle input, what is the fewest number of steps to go from e to the medicine molecule?


In [1]:
filepath = "..\\data\\input_day_19.txt"
test1 = "..\\test\\test19_1.txt"
test2 = "..\\test\\test19_2.txt"
test3 = "..\\test\\test19_3.txt"
test4 = "..\\test\\test19_4.txt"

In [2]:
# first we import our files
def read_input(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    return lines

In [3]:
def convert_input(lines):
    
    molecule = lines[-1].strip()
    replacements = dict()
    replacements_inv = dict()
    
    for line in lines[:-2]:        
        original, _ , new = line.split()
        
        if original in replacements:
            replacements[original].append(new)
        else:
            replacements[original] = [new]
            
        if new in replacements_inv:
            replacements_inv[new].append(original)
        else:
            replacements_inv[new] = [original]
            
    return molecule, replacements, replacements_inv

In [4]:
def convert_input_two(lines):
    
    molecule = lines[-1].strip()
    replacements_inv = dict()
    
    for line in lines[:-2]:
        original, _ , new = line.split()
        
        #if original in replacements_inv:
        #    replacements_inv[new].append(original)
        #else:
        replacements_inv[new] = original
        
               
    return molecule,  replacements_inv

In [5]:
def perform_replacement(original, position, replacement, double=False):
    '''
    Performs the desired replacement at the position and returns the new molecule
    example:
    original = HOH, position = 0, replacement: H => HO
    returns
    HOOH
    '''
    if double:
        if position != len(original) - 1:
            original = original[:position]+"x"+original[position+2:]
        else:
            original = original[:position] + "x"
    if position != len(original) - 1 :
        return original[:position] + replacement + original[position+1:]
    return original[:position] + replacement 
    

In [6]:
def molecule_viable(molecule, target_molecule):#, not_viable):
    
    '''
    Ar is a product that keeps accumulating in our molecule and does not gets replaced. 
    This makes Ar a target to check if the molecule can still become our target molecule.
    '''
    length_viable = len(molecule)<len(target_molecule)
    Ar_viable = molecule.count("Ar") <= target_molecule.count("Ar")
    Rn_viable = molecule.count("Rn") <= target_molecule.count("Rn")
    Y_viable  = molecule.count("Y") <= target_molecule.count("Y")
    return length_viable and Ar_viable and Rn_viable #and (molecule not in not_viable)

In [7]:
def day19a(filepath):
    
    # read the input and extract the original molecule and the replacements performed
    lines = read_input(filepath)
    molecule, replacements, _ = convert_input(lines)
    
    # use a set so we don't have copies of molecules
    new_molecules = set()
    
    for i, atom in enumerate(molecule):
        # check the one letter atoms
        if atom in replacements.keys():
            for new_atom in replacements[atom]:
                new_molecule = perform_replacement(molecule, i, new_atom)
                new_molecules.add(new_molecule)
        # check the two letter atoms
        if i!= len(molecule)-1:
            atom_pair = atom+molecule[i+1]
            if atom_pair in replacements.keys():
                for new_atom in replacements[atom_pair]:
                    new_molecule = perform_replacement(molecule, i, new_atom, double=True)
                    new_molecules.add(new_molecule)
    
    print(f"We get {len(new_molecules)} distinct molecules by performing one atom replacements.")
    #print(new_molecules)
    return len(new_molecules)

In [8]:
def day19b(filepath):
    
    import random
    
    # read the input and extract the original molecule and the replacements performed
    lines = read_input(filepath)
    target_molecule, replacements, replacements_inv = convert_input(lines)
    
    # Start with all possible products from the electron
    current_gen = set(replacements["e"])
    operation_count = 1
    target_length = len(target_molecule)
    #not_viable = set()
    
    for i in range(100):
        print(i)
        operation_count += 1
        new_gen = set()
        for molecule in current_gen:
            for i, atom in enumerate(molecule):
                # check the one letter atoms
                if atom in replacements.keys():
                    for new_atom in replacements[atom]:
                        new_molecule = perform_replacement(molecule, i, new_atom)
                        if molecule_viable(new_molecule, target_molecule):#, not_viable):
                            new_gen.add(new_molecule)
                        #else:
                        #    not_viable.add(new_molecule)
                # check the two letter atoms
                if i!= len(molecule)-1:
                    atom_pair = atom+molecule[i+1]
                    if atom_pair in replacements.keys():
                        for new_atom in replacements[atom_pair]:
                            new_molecule = perform_replacement(molecule, i, new_atom, double=True)
                            if molecule_viable(new_molecule, target_molecule):#, not_viable):
                                new_gen.add(new_molecule)
                            #else:
                            #    not_viable.add(new_molecule)
                
                if new_molecule == target_molecule:
                    return operation_count
        current_gen = new_gen

In [9]:
def reverse_processing(molecule, target_molecule, replacements_inv):
    ''' 
    This is based on the solution by what-a-baller on the code of advent subreddit, I'm not sure how this process guarantees 
    an optimal solution.
    '''
    count = 0
    old_molecule = ""
    end_products = list(replacements_inv.keys())
    
    while old_molecule != molecule:
        old_molecule = molecule[::]
        for ep in end_products:
            while ep in molecule:
                count += molecule.count(ep)
                molecule = molecule.replace(ep, replacements_inv[ep])
    return (molecule==target_molecule)*count
            
   


In [19]:
def day19b_v2(filepath):

    import random

    # read the input and extract the original molecule and the replacements performed
    lines = read_input(filepath)
    molecule, replacements_inv = convert_input_two(lines)
    target_molecule = "e"
    
    steps = 0
    while steps==0:
        steps = reverse_processing(molecule, target_molecule, replacements_inv)
    print(f"The medicine can be made in {steps} steps.")

In [11]:
day19b_v2(filepath)

The medicine can be made in 200.


In [12]:
def test19a():
    # Test perform_replacement with the example given
    original_test = "HOH"
    assert perform_replacement(original_test, 0 , "HO") == "HOOH"
    assert perform_replacement(original_test, 2 , "HO") == "HOHO"
    assert perform_replacement(original_test, 0 , "OH") == "OHOH"
    assert perform_replacement(original_test, 2 , "OH") == "HOOH"
    assert perform_replacement(original_test, 1 , "HH") == "HHHH"
    print("Passed all replacement checks")
    
    # test the entire function on our example
    assert day19a(test1) == 4 # we get 4 distinct molecules
    assert day19a(test2) == 8
    print("Passed all checks")

In [13]:
def test19b():
    
    # from the example given
    assert day19b(test3) == 3
    assert day19b(test4) == 6
    
    print("Passed all checks")

In [14]:
test19a()

Passed all replacement checks
We get 4 distinct molecules by performing one atom replacements.
We get 8 distinct molecules by performing one atom replacements.
Passed all checks


In [15]:
test19b()

0
1
0
1
2
3
4
Passed all checks


In [16]:
day19a(filepath)

We get 518 distinct molecules by performing one atom replacements.


518

In [18]:
day19b_v2(filepath)

The medicine can be made in 200.
